In [8]:
from database import engine
from model import Base
Base.metadata.create_all(bind=engine)

ModuleNotFoundError: No module named 'Docling_Agent'

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from tools.retriver_tool import create_retriever_tool
from tools.document_loader import upsert_text
from tools.pdf_tool import upsert_pdf
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()    
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API")

In [ ]:
document_retriever = create_retriever_tool(
    index_name="db",
    tool_name="document_retriever",
    description="Search knowledge base for document content relevant to a query"
)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional document analyst assistant. Use the document_retriever tool to find relevant information before answering questions. Always cite sources using metadata."),
    ("human", "Question: {input}\n\nContext from documents:\n{context}"),
    ("human", "Based on the above context, provide a comprehensive answer to the question. Cite sources using [Source: filename, Page #] when available.")
])
llm = ChatGroq(model="qwen-qwq-32b")

retrieval_chain = (prompt, llm)

agent_executor = AgentExecutor(
    agent=retrieval_chain,
    tools=[document_retriever],
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
def process_and_query(query: str, directory: str = None) -> str:
    """
    Process documents if provided, then execute a query against the knowledge base
    
    Args:
        query: Natural language question
        file_path: Optional document to process before querying
        
    Returns:
        LLM response with sourced references
    """
    if file_path:
        print(f"Processing document: {file_path}")
        if file_path.endswith('.pdf'):
            upsert_pdf(file_path)
        elif file_path.endswith('.txt'):
            upsert_text(file_path)
        print("Document processing complete. Index updated.")
    
    result = agent_executor.invoke({"input": query})
    return result



In [ ]:
response = process_and_query("What are the key safety requirements?", "/home/thkr1408/Projects/Inquiro/uploads")
print(response)